# Forecasting

- Statistical and machine learning models on time-series data
- Credits: DSCI 574 Spatial & Temporal Models, Quan Nguyen, Feb 2022
    - Which is adapted from DSCI 574 by Tomas Beuzen, Feb 2022
- Detailed local version: [Note 1](http://localhost:8888/doc/tree/mds/Block5/simon-block5/574-Note-1.ipynb) and [Note 2](http://localhost:8888/doc/tree/mds/Block5/simon-block5/574-Note-2.ipynb)

## Intro to Time-Series

### What is time series?

- A time series is a collection of observations recorded sequentially in time
    - For the statistically inclined, we define a time series as a collection of random variables indexed by time
        - For example, consider the sequence of random variables, $y_1, y_2, y_3$ etc., where the random variable $y_i$ denotes the value of the times series at the $i$th time point.
    - In general, a collection of random variables, $y_t$ indexed by $t$ is referred to as a *stochastic process*.
- Observations in a series may be evenly spaced (**regular time series**) or unevenly space (**irregular time series**)
    - We will be focusing on regular time series in this course
    - If you encounter an irregular time series, typically you could aggregate it to a regular interval, and/or to impute missing values
- Generally there are two main things we want to do with a time series:
    1. Explanatory modelling to **understand the past**
    2. Predictive modelling to **forecast the future**
    - (In this section we will be focusing on the explanatory modelling to understand the past)

### Time Series Features

Visualization and temporal dependency:

- The key difference between other data and time series data: temporal dependency!
    - We can quantify this dependency by looking at the correlation of a time series with "lagged" values of itself. 
        - We call this **autocorrelation**
    - We can easily "lag" a time series in Pandas with the `.shift()` method: 
        - `df["time (lag=1)"] = df["time"].shift(1)`
        - Correlation between the two: `df["time"].corr(df["time (lag=1)"])`
- A **correlogram** plots the autocorrelation function (ACF) on the y-axis and lags on the x-axis 
    - (we call it the autocoreelation *function* because it is a function of lag)
    - useful package and functions: `from statsmodels.graphics.tsaplots import plot_acf`
- We'll explore this notion of trends more later, but for now, some key observations about the correlograms:
    - The ACF will almost always decay with the lag (observations farther apart in time are less correlated)
    - If a series alternates (i.e., consecutive values tend to be on the opposite sides of the mean, like our sunspots data), then the ACF alternates too.
    - If a series has seasonal or cyclical fluctuations, the ACF will oscillate at the same frequency.
    - If the series has a trend, the ACF will have a very slow decay due to high correlation of the consecutive values (which tend to lie on the same side of the mean)
    - In general, experience is required to glean much from an ACF plot. We will use the correlogram as a model selection tool later in the course.
    
Time Series Pattern:
- There are 3 main patterns of a time series you should be aware of:
    1. **Trend**: long term increases or decreases in the series.
    2. **Seasonality**: regular variation in the series at some fixed interval, e.g., month, day of week, time of day, etc.
    3. **Cyclicity**: variations in the series that repeat with some regularity but of unknown and changing period.

White noise:
- Time series that show no autocorrelation with zero mean and constant variance are called **white noise**
    - often we assume that the white noise is iid and Gaussian distributed denoted as $w_t\sim\mathcal{N}(0,\sigma^2)$
- Think of white noise as completely uninteresting with no predictable patterns
    - if our series is white noise, it means it is a series of random numbers and cannot be predicted
    - white noise can also help us to check whether there is information/dependency in our time series that we can model
        - (as we will see in the next section)
- As a result, we expect the ACF of a white noise series to be close to 0 (no correlation) for all lags

### Time Series Decomposition

- When we decompose a time series, we usually split it into 3 components:
    1. **Trend-cycle** ($T$)
        - Comparing to seasonal component, trend component is a long term change in the mean of the series, whereas the seasonality is a regular, repeating variation that repeats at known periods
        - **Curve-fitting and moving average**
    2. **Seasonal** ($S$)
        - The difference between "seasonality" and "cyclicity" in a time series is that
            - Seasonality is a regular, repeating variation that repeats at known periods
            - Cyclicity is a repeating variation of varying period and magnitude
        - Subtracting/Dividing the trend-cycle effect(i.e. detrending) first then take average
    3. **Remainder** ($R$) (also called the "residual")
        - Calculate by following the formulae below (use observation to subtract or divide $S$ and $T$)
- There are two main ways we can combine/decompose these components to make up a time series:
    1. **Additive**: $y_t = S_t + T_t + R_t$. 
        - Appropriate if the magnitude of the seasonal fluctuations, or the variation around the trend-cycle, does not vary with the value of the series.
    2. **Multiplicative**: $y_t = S_t \times T_t \times R_t$. 
        - Appropriate if variation in the seasonal pattern, or the variation around the trend-cycle, appears to be proportional (i.e. variation increases as the time goes by) to the value of series.
    - Usually we would use additive method since multiplicative one has a relatively strong assumption on the proportional relationship between time and the variation in the seasonal pattern.
- `statsmodels.tsa` has many great tools that handle decomposition for us
- Probably the most common is an STL decomposition which uses Loess (kind of like kNN and OLS regression had a baby)

## Intro to Forecasting

- Unlike predicting, forecasting means that we are predicting values in the future.

### Baseline methods

Before we get into each method, some notations:
- $y_t$: value of a time series at time $t$
- $h$: a forecast horizon
    - i.e. $h=1$ means we want to predict one time step ahead
- $T$: length of a time series
- $\hat{y}$: forecast value
- $y$: observed value
- $\hat{y}_{t|t-1}$: the value of $\hat{y}_t$ given $y_{t-1}$

Now the baseline methods:
- **Average**
    - Use the average of the series for all future forecasts
    - i.e. mathematically $\hat{y}_{T+h}=\bar{y}$ for all $h$
- **Naive**
    - Use the last observation for all forecasts
    - i.e. $\hat{y}_{T+h|T}=y_T$ for all $h$
- **Seasonally-adjusted naive**
    - Similar to naive method, but the data are seasonally adjusted by applying a classical decomposition
    - i.e. $\hat{y}_{T+h|T}=y'_T$ for all $h$ where $y'$ stands for the transformed seasonally adjusted data
        - e.g. if we are using multiplicative decomposition, then $y' = \frac{y}{\text{model.seasonal}}$
- **Seasonal naive**
    - Set each forecast as the last observed value from the same season of the year (e.g. the same month of the previous year)
    - More specifically for our monthly data, the forecasts for all future January is the last observed January value
- **Drift**
    - Forecasts equal to last value in the series plus the average (global, not step-wise) change of the series
    - i.e. $\hat{y}_{T+h|T}=y_T+h\left(\frac{y_T-y_1}{T-1}\right)$

### Exponential models

- In Simple Exponential Smoothing(**SES**), our forecast is an exponentially weighted average of past values:
$$\hat{y}_{t+1} = \alpha{}y_t + \alpha{}(1 - \alpha{})y_{t-1} + \alpha{}(1 - \alpha{})^2y_{t-2} + \cdots$$
    - where $0\le\alpha{}\le1$ and $\hat{y}$ refers to a forecasted value
    - We can re-write that in the recursive form:
$$\hat{y}_{t+1|t} = \alpha{}y_t + (1-\alpha{})\hat{y}_{t|t-1}$$
    - (TODO)
- In **Holt's method**, we extend smoothing based on the SES:
$$\hat{y}_{t+h|t}=\ell_t+hb_t$$

$$\ell_t=\alpha{}y_t+(1-\alpha{})(\ell_{t-1}+b_{t-1})$$

$$b_t=\beta(\ell_t-\ell_{t-1})+(1-\beta)b_{t-1}$$
    - We now have two key parameters $\alpha$ (to control smoothness of the level) and $\beta$ (control smoothness of trend). Read more about it [here](https://otexts.com/fpp3/holt.html) if you wish.
    - All we're doing here is forecasting the next values as an exponentially weighted average of past values and past trend
- **Holt-Winter's method** extends even further:
$$\hat{y}_{t+h|t}=\ell_t+hb_t+s_{t+h-m(k+1)}$$

$$\ell_t=\alpha{}(y_t-s_{t-m})+(1-\alpha{})(\ell_{t-1}+b_{t-1})$$

$$b_t=\beta(\ell_t-\ell_{t-1})+(1-\beta)b_{t-1}$$

$$\text{additive: }s_t=\gamma(y_t-\ell_{t-1}-b_{t-1})+(1-\gamma)s_{t-m}$$ 
$$\text{multiplicative: }s_t=\gamma\frac{y_t}{\ell_{t-1}+b_{t-1}}+(1-\gamma)s_{t-m}$$

### ETS models
- The methods above providing point forecasts, but what if we want to have prediction intervals?
- The generalization of exponential smoothing algorithms to statistical models that model distributions, include an error term, and can generate prediction intervals are known as **ETS** models (**E**rror, **T**rend, **S**easonal), where:
    - E = {additive, multiplicative}
    - T = {none, additive, additive damped}
    - S = {none, additive, multiplicative}
- You can read more about ETS models and their derivation [here](https://otexts.com/fpp3/ets.html) and in [Appendix B](https://pages.github.ubc.ca/MDS-2021-22/DSCI_574_spat-temp-mod_students/lectures/appendixB_state-space-models.html) but their derivation is not really important to know and beyond the scope of this course.
    - However do note that an ETS model will usually give the same/similar results to the algorithms above, but with the added bonus of being able to generate prediction intervals
    - In Rob Hyndman's [own words](https://robjhyndman.com/hyndsight/estimation2/) *"the results from ETS are usually more reliable (than the algorithmic exponential models)"*
- Rather than optimizing based on minimizing the SSE, ETS models optimize by maximizing the likelihood (we assume errors are normally distributed).
    - For more details on that also see [Appendix B](https://pages.github.ubc.ca/MDS-2021-22/DSCI_574_spat-temp-mod_students/lectures/appendixB_state-space-models.html)
|Trend Component|Seasonal Component|
|---|---|
|None `(N)`|None `(N)`|
|Additive `(A)`|Additive `(A)`|
|Additive damped `(Ad)`|Multiplicative `(M)`|

|Notation|Method|
|---|---|
|`(N,N)`|Simple exponential smoothing|
|`(A,N)`|Holt's method|
|`(A,A)`|Additive Holt-Winter's method|

### Selecting a model

#### In-sample methods
- Metrics
    - The most common metrics are:
        - Akaike information criterion(AIC)
        - Bayesian information criterion(BIC)
        - ![](../images/aic_bic.png)
        - Sum of Squared Errors (SES)/Mean Squared Errors (MSE)/Root Mean Squared Errors(RMSE)
    - We can extract them from most models in `statsmodels` with `model.summary()` or extract with `model.aic/model.bic/model.mse` 
- Residuals
    - We can use residuals to reflect how well our model captures information in the data by
        1. Visual inspection (residuals are uncorrelated, have zero mean, and ideally normally distributed)
            - We can use `plot_acf` on `model.resid` and see if residuals are significantly different from white noise
            - If it has structure and different from noise, then it is not good
        2. Running diagnostic Portmanteau tests (e.g. Ljung-Box-Perce test, etc.)
            - The *[Ljung–Box test](https://en.wikipedia.org/wiki/Ljung%E2%80%93Box_test)* tests whether a group of autocorrelations is significantly different from white noise
            - The *[Jarque-Bera test](https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test)* tests whether residuals are significantly different from a normal distribution (based on skewness and kurtosis)
            - `statsmodels` provides the test statistics for us with `model.test_serial_correlation('ljungbox', lags=24)` and `model.test_normality('jarquebera')`
    - However these tests tend to be not that useful in practice... but could consider them since they are included in the most packages anyway
- See examples with code details on the [course note section](https://pages.github.ubc.ca/MDS-2021-22/DSCI_574_spat-temp-mod_students/lectures/lecture2_intro-to-forecasting.html#in-sample-methods)

#### Out-of-sample methods
- Usually we are interested in using models to forecast, hence we care about its performance on unseen data
- Typically method would be having a training set and a validation set
- To measure the performance of model forecasts, the most common regression metrics are:
    1. Mean Absolute Error (MAE): $\frac{1}{n}\sum^{n}_{i=1}|y_i - \hat{y}_i|$
    2. Root Mean Squared Error (RMSE): $\sqrt{\frac{1}{n}\sum^{n}_{i=1}(y_i - \hat{y}_i)^2}$
    3. Mean Absolute Percentage Error (MAPE): $\frac{1}{n}\sum^{n}_{i=1}|\frac{y_i - \hat{y}_i}{y_i}|$
    4. Mean Absolute Scaled Error (MASE): $\frac{MAE}{\frac{1}{T-1}\sum^{T}_{t=2}|y_t-y_{t-1}|}$
- Some notes on these metrics:
    - MAE and RMSE are popular in practice because they are easier to interpret. Closer to 0 is better.
    - MAPE is scale-free and aims to proportionalize errors, such that the error for $\hat{y}=12$ and $y=10$ (MAPE = 20%, MSE = 4) is the same as $\hat{y}=120$ and $y=100$ (MAPE = 20%, MSE = 400).  Closer to 0 is better. 
        - MAPE is problematic if 0 values are expected (divide by 0 error) and it is also not symmetrical, i.e., $\hat{y}=150$ and $y=100$ gives $MAPE=\frac{|100-150|}{100}=33.33\%$, but $\hat{y}=100$ and $y=150$ gives $MAPE=\frac{|150-100|}{150}=50\%$. 
        - There is a version, `sMAPE` available that is symmetrical, but MASE is often preferred.
    - MASE scales the MAE based on the MAE of a naive forecast on the training data. I think of this as the "r-squared" of the forecasting world. It corrects the above errors. Value < 1 indicate forecasts are better than in-sample naive forecasts.

## ARIMA Models

### Stationary
- A stationary time series is one whose properties do not depend on time
    - Is roughly horizontal
    - Has a constant mean & variance
    - Does not show predictable patterns (e.g., seasonality)
    - Note that a time series can be non-stationary even if it has no trend
        - The expected value of a series may depend on time as a result of seasonality and changing variance
    - On the other than, a time series can be stationary even if it has non-zero autocorrelation for some lags higher than 0
        - A series can be stationary, yet autocorrelation can still arise when observations are influenced by previous observations; consider a stationary AR(1) process as an example
- How can we make a time-series more stationary?
    - Transformation (log-return)
$$ log return = log(\frac{y_t}{y_{t-1}}) = log(y_t) - log(y_{t-1}))$$
        - commonly used for financial data
    - Differencing: calculate the differences between consecutive observations in the series
        - **Differencing can help stabilise the mean of a time series by removing changes in the level of a time series, and therefore eliminating (or reducing) trend and seasonality.**
        - 1st order differencing: 
$$ y'_t = y_t - y_{t-1} $$
        - 2nd order differencing: 
$$ y''_t = y'_t - y'_{t-1} $$
$$ = y_t - 2y_{t-1} + y_{t-2} $$
    - See [code example](https://pages.github.ubc.ca/MDS-2021-22/DSCI_574_spat-temp-mod_students/lectures/lecture3_arima-models.html#how-to-make-time-series-data-more-stationary) on how we make the time-series stationary with Bitcoin and CO2 data separately
        - Notice in the CO2 data, we encouter the case where the (first) differenced data is still not appear to be stationary; a couple of methods we could use here
            1. Differencing again (this is called "second-order differencing")
            2. Seasonal differencing (the difference between an observation and the previous observation from the same season)
            3. Applying a transform (when there is changing variance in our series, we might want to apply a transform like a log, or Box-Cox transform)

### ARIMA Models

AR: AutoRegressive model, regression of the variable against itself; we simply forecast with a linear combination of past values:
$$y_t = c + \alpha_1y_{t-1} + \alpha_2y_{t-2} + ... + \alpha_py_{t-p} + \epsilon_t$$

MA: Moving Average, uses past forecast errors in a regression-like model:
$$y_t = c + \epsilon_t + \beta_1\epsilon_{t-1} + \beta_2\epsilon_{t-2} + \cdots + \beta_q\epsilon_{t-q}$$


ARMA: Autoregression + moving average

ARIMA: Autoregressive Integrated Moving Average  (ARMA + differencing)

Seasonal ARIMA (SARIMA) and adding explanatory to the ARMA (SARIMAX)

### Choosing Order
- Picking `p` for the AR component
    - Recall that a corellogram (ACF plot) shows autocorrelations at different lags
        - however ACF only tells us how correlated it is between $y_t$ and $y_{t-h}$
            - The problem is that if $y_t$ and $y_{t-1}$ are correlated, then $y_{t-1}$ and $y_{t-2}$ are also correlated, and therefore, $y_t$ and $y_{t-2}$ have indirect correlation (via $y_{t-1}$).
            - This makes it hard to isolate exactly which lags are important in our series. We can clearly see this in the model above which should only show one significant lag.
    - The solution to the problem with ACF is **partial autocorrelation** (PACF) which removes intermediate effects between $y_t$ and $y_{t-h}$. 
        - The partial autocorrelation is estimated as the last coefficient in an autoregressive model. So $PACF(k)$ is the $k$th estimated coefficient in an `AR(k)` model
    - By look at the correllogram with PACF, we can pick the `p` (the order of AR model) where `p` equals to the number of lags it takes to decrease down to the confidence interval range
        - we want to see how many lags are heavily correlated
- Picking `q` for the MA component
    - Recall that `MA(q)` models are based on white noise. 
    - As we saw before, the value of an `MA(q)` model at time $t$ is a weighted sum of the last `q` values of the white noise process. 
    - Since there is no dependence structure on values at lags higher than `q`, it's just white noise, so we would expect the ACF to "cut off" at lag `q`, and pick that `q`
- See [coded example](https://pages.github.ubc.ca/MDS-2021-22/DSCI_574_spat-temp-mod_students/lectures/lecture3_arima-models.html#choosing-orders) in the course notes.
    - Notice that in the sliding bar example with phi (the correlation coefficient $\phi$), when we have high phi, we tend to have larger correlation and we can see graduate decrease in the ACF and quick reduce in PACF with some bouncing around the zero line

|Model|ACF|PACF|
|---|---|---|
|MA(q)|Cuts-off at lag `q`|Tails off, no pattern|
|AR(p)|Tails off (exponentially or like a "damped" sine wave)|Cuts-off at lag p|

- Auto-arima
- Box-Jenkins
![](../images/box_jenkins.png)

## Forecasting with ML

### Machine learning processes
- Data wrangling
    - We might only have one single column of the time series, and it can be helpful to have features that are lagged versions of the given time series data
    - The way to do this with Pandas we could use `.shift()`
- Splitting into train, validation and test sets
    - One important difference from other data is that for time series we almost **NEVER shuffle the data**
        - Why? Because this would disrupt the temporal order of the data and can leak information from the test/validation sets into the training set
    - In terms of code, we would use the `train_test_split` function with `shuffle=False`
    - For validation data, it is similar to splitting train and test, but there are two ways of creating each split for cross-validation:
        - **Expanding window** approach: always start from the starting point of the time series, but take different length of time data; in this way each split would share common data; but the size of validation data is fixed
        - **Fixed/Sliding window** approach: keep the split size/window width to be the same per each split/window; in this way each split would not have the same data without any overlapping
        - There is a trade-off to think about: 
            - Fixed (smaller) windows have less computation but may result in poorer parameter estimates since less data being used for training
            - Expanding windows use more data to estimate parameters but data way in the past may not be as representative as current (and future data)
        - In sklearn we have the function `TimeSeriesSplit` does the job which produces the results/objects that `cross_val_score` function accepts
            - Notice we could use `max_train_size` argument to force the fixed window size in the `TimeSeriesSplit` function
    - Another important difference is varying length of each split. For applying standard cross-validation with other data we only have fixed split, but as we see above for time series version, due to the fact that it cannot be shuffled, it might (often it does) have the expanding windows/splits with different sizes of splits and one split contains another.
        - Since for time series, it would only make sense to forecast the future, not the past (or current I suppose)
- Building the model
    - This is more straight-forward where we just build the model with the Python packages
- Comparing to statistical model like ETS or ARIMA, ML approach might be bad since
    - (Con) More data wrangling required
    - (Con) Relatively more hyperparameters to tune
    - (Con) Higher computational cost and might be slower

### Forecasting Step in ML

- One-step forecasts
    - Takes current and previous data to predict the next value (and next value only)
    - For example, based on the sales today, yesterday and the day before and use that data to predict tomorrow's sales
        - in terms of the code, we could have `model.fit(X_train.iloc[train], y_train.iloc[train])` then predict with `model.predict(X_train.iloc[valid])`
- Multi-step forecasts
    - E.g. we have monthly data and we want to forecast our time series 12 months ahead into the future
    - There are four main ways we can approach this problem
        1. Recursive forecasts
            - As we forecast into the future, the prediction at time $t$ will become an input to the model for predicting at time $t+1$
            - The idea is basically iterate through the length of prediction/forecast index/timestamps, while making sure for each iteration everything before the current time is included
            - Notice we are not retraining the model but using the same model for prediction at each step
            - Putting into a function we have:
```Python
            def recursive_forecast(input_data, model, n=20, responses=1):
                forecast = np.empty((n, responses))  # where we'll store our forecasts
                for i, n in enumerate(range(n)):     # loop for making forecasts one at a time
                    forecast[i] = model.predict(input_data.reshape(1, -1))  # model forecast
                    input_data = np.append(forecast[i], input_data[:-responses])  # append forecast to input data for next forecast
                return forecast.reshape((-1, responses))
```
            - (Pro) easy to implement and only requires one single model
            - (Con) the performance might not be good due to the accumulated errors
        2. Direct forecast
            - In the direct approach we make a model for every forecast horizon we wish to predict
            - It has much higher computational cost and maintenance cost (need one model for every forecast horizon, we are building a new model for each $t+i, i=1,2,\cdots$), but it does not propagate uncertainty like the recursive method
            - (Pro) typically more accurate prediction since we do not require recursion and errors do not accumulate
            - (Con) need one model per forecast horizon, extra effort to develop and maintain
        3. Hybrid
            - The idea is to combine the previous two; a separate model is developed for each forecast horizon, but each model can incorporate the predictions of the previous timestamp
            - Mathematically, 
            $$\hat{y}_{t+1} = model_1(y_t, y_{t-1}, y_{t-2}, \cdots)$$ 
            $$\hat{y}_{t+2} = model_2(\hat{y}_{t+1}, y_{t}, y_{t-1}, \cdots)$$ 
            $$\hat{y}_{t+3} = model_2(\hat{y}_{t+2}, \hat{y}_{t+1}, y_{t}, \cdots)$$
            - Obviously takes more care and thought to set up and often does not improve that much...
        4. Multi-output
            - In this approach, we develop a model that makes multiple predictions at once
            - For example, a model that spits out predictions for $t+1$ and $t+2$

### Feature preprocessing and Feature engineering
- Feature preprocessing
    - Common feature preprocessing techniques:
        - Coerce to stationary (through differencing or transforms)
            - many ML models (especially tree-based models) find it difficult to model non-stationary data
            - while we can often try to account for seasonal cycles by using lagged features, or time stamp features, trends can be harder to model
            - linear models may do better in these situations, but usually we can just make the data stationary
            - check out the [example in the course note](https://pages.github.ubc.ca/MDS-2021-22/DSCI_574_spat-temp-mod_students/lectures/lecture4_machine-learning.html#pre-processing) on how to do so with a `FunctionTransformer` from sklearn
        - Smoothing (usually with a moving average)
        - Removing outliers (more about this in next section)
- Feature engineering
    - Common feature engineering:
        - Lagging response/features (we have seen this already)
        - Adding time stamps (e.g., month of year, day of week, am/pm, etc.)
            - See an [example on adding time stamps](https://pages.github.ubc.ca/MDS-2021-22/DSCI_574_spat-temp-mod_students/lectures/lecture4_machine-learning.html#feature-engineering) in the course notes
        - Rolling statistics (e.g., rolling mean, median, variance, etc)
            - The previous example on adding time stamps also include the rolling statistics after the time stamps part

### Multivariate time series
- Most of the concepts we just discussed extend to the multi-variate case
    - You just need to use a model that can produce multiple outputs (e.g., decision tree, random forest, KNN, neural network, etc.)
- We could also see the application with [`VAR` in the `statsmodels`](https://www.statsmodels.org/dev/vector_ar.html)

### How do we know a time series is useful/useless?
- Visually check if the model performs well by plots
- Check R-squared values and mean squared error
    - Recall the objective of forecasting: we are not predicting, we just need the the returns of the stock/securities you're trading (i.e. is it going up or down?) instead of the actual price
- Sometimes our time series is just a series of random walk where forecast just would not work
    - There are probably just no structure to learn (hence the time series is sort of useless to make any forecasting work)
- This is why we have baseline models in time-series forecasting
    - Recall the baseline models from Lecture 2:
        - Naive
        - Seasonal naive
        - Drift
    - Baseline forecasts quickly flesh out whether you can do significantly better.
    - If you can’t, you’re probably working with a random walk.
        - A random walk is one in which future steps or directions cannot be predicted on the basis of past history. When the term is applied to the stock market, it means that short-run changes in stock prices are unpredictable.

## Other Forecasting Techniques

### Forecast with Uncertainty

- How do we forecast with a "forecast interval" similar to a prediction/confidence interval?

#### Analytical
- If we assume the distribution of forecasts are normal, then the prediction interval at the $h$-step forecast is simply:
$$\hat{y}_{T+h|T}\pm{}c\times\hat{\sigma}_h$$
    - where $\hat{\sigma}_h$ is the estimated s.d. of the $h$-step forecast distribution
    - and $c$ determines the coverage probability
        - usually we use $c = 1.96$ which gives us a 95% prediction interval
- The main focus here is estimating $\hat{\sigma}_h$ and for one-step ahead forecasts (i.e. $h=1$) it is often reasonable to estimate it as: $$\hat{\sigma}_{1}=\sqrt{\frac{1}{T-K}\sum^{T}_{t=1}e^{2}_{t}}$$
    - where $K$ is the number of parameters, $T$ is the total length of the time series and $e$ is the fitted model residuals
- For multi-step forecasts (i.e. $h$ gets larger), things get more difficult, and prediction intervals are typically wider as the forecast horizon increases.
    - at the same time, if we have longer length of the time series, we might have less uncertainty and the prediction intervals would be narrower
- Some methods have been derived mathematically as follows:

|Method|Forecast standard deviation|
|---|---|
|Mean|$\hat{\sigma_h}=\hat{\sigma}_{1}\sqrt{1+\frac{1}{T}}$|
|Naïve|$\hat{\sigma_h}=\hat{\sigma}_{1}\sqrt{h}$|
|Seasonal naïve|$\hat{\sigma_h}=\hat{\sigma}_{1}\sqrt{\frac{h-1}{m}+1}$|
|Drift|$\hat{\sigma_h}=\hat{\sigma}_{1}\sqrt{h(1+\frac{h}{T})}$|
|ETS|Chapter 6 of [Forecasting with Exponential Smoothing](https://robjhyndman.com/expsmooth/)|
|ARIMA|Section 6.4 of [Introduction to Time Series and Forecasting](https://www.springer.com/gp/book/9783319298528)|

#### Simulation and Bootstrapping
- If it is not reasonable to assume a normal distribution for future forecasts and/or there are no analytical solutions, the most common alternative is to use bootstrapping to generate prediction intervals
    - The bootstrap method is a resampling technique used to estimate statistics on a population by sampling a dataset with replacement. It can be used to estimate summary statistics such as the mean or standard deviation
- When is the bootstrapping method better than assuming a normal distribution?
    - If it is unreasonable to assume a normal distribution for future forecasts
    - If you do not want to assume a distribution
    - If there are no analytical solutions for determining standard error evolution into the future.
- Instead, we assume future errors will be similar to past errors
    - Randomly draw from past errors and add them on to forecasts to simulate future values
    - Simulate over and over to generate many realization of future scenarios and then calculate prediction intervals by calculating percentiles on these realizations.
    - For example, a forecast error is defined as: $$\epsilon_t = y_t - \hat{y}_{t|t-1} $$
        - We can re-write this as $$ y_t = \hat{y}_{t|t-1} + \epsilon_t  $$
        - We can simulate the next observation using $$ y_{T+1} = \hat{y}_{T+1|T} + \epsilon_{T+1}  $$
            - where $\hat{y}_{T+1|T}$ is the one-step forecast and $\epsilon_{T+1}$ is the unknown future error.
        - Assuming future errors will be similar to past errors, we can replace $\epsilon_{T+1}$ by sampling from the collection of errors we have seen in the past (i.e. the residuals)
- In terms of code, `statsmodels` implementations of `ARIMA` and `ETS` has the `simulate()` method does this simulation
    - We can use the dataframe `.quantile()` method to calculate desired quantiles (e.g. 97.5 and 2.5 for 95% prediction interval) from all these simulations

#### Quantile Regression

- In quantile regression our parameter of interest is no longer usual mean, but we wish to predict the particular quantile of a distribution
- The idea here is that we are predicting some quantile of a future time step, say $q=0.9$. That is, we expect the actual value to be below this quantile 90% of the time, and above it 10% of the time.
- We need to account for the fact that the true value is more likely to be below than above our prediction but weighting under/over prediction differently and that's what the quantile loss is doing
    - For this, we often use the "quantile loss" (sometimes called "pinball loss"):
$$
\mathcal{L}=
     \left\{
\begin{array}{ll}
      (q-1)(\hat{y}_{t,q}-y_t) \text{,} \;\; \text{ if } y_t < \hat{y}_{t,q} \\
      q(y_t-\hat{y}_{t,q}) \text{,} \;\;\;\;\;\;\;\;\;\; \text{ if } y_t \ge \hat{y}_{t,q} \\
\end{array} 
\right.
$$
- Different packages have quantile loss implemented in various forms, e.g., the `statsmodels` class `QuantReg` or the `sklearn` class `GradientBoostingRegressor`; one might consider to use `lightgbm` (light gradient boosting machine) for quantile regression
- We could also implement this loss easily in a neural network framework like `pytorch`
    - Quantile loss is not currently a supported criterion in `pytorch` but it's easy to define ourselves. We really have two options:
        1. Train a network for each quantile we want to predict; or
        2. Train a network to output multiple quantiles at once
- Quantile regression versus Bootstrapping
    - Quantile regression models a specific quantile. Model parameters are learned to make forecasts of this quantile. Typically, one model is required per forecast.
    - Bootstrapping simulates realisations of future series by bootstrapping from residuals at each time step. We then determine intervals by calculating quantiles from all these simulations.
- Quantile regression pro and con:
    - (Pro) it models the quantiles directly, does not assume a distribution
    - (Con) it typically needs one model per quantile, need enough data to accurately estimate quantiles

#### Evaluating Distributional Forecast Accuracy

- Problem: prediction intervals for time series forecasts are almost always too narrow
    - Because there are many uncertainty and there are 4 main sources of uncertainty:
        1. Random error term;
        2. Uncertainty of model parameter estimates;
        3. Uncertainty introduced by choice of model;
        4. Uncertainty about the consistency of the data generating process into the future
    - Most of the methods above only account for source 1
    - Unlike OLS regression, there is no closed form solutions available to include source 2
    - Simulation tries to account for source 2 and source 3 to some extent
    - But source 4 is practically impossible to know and can only be guessed at
- How do we evaluate our distributional forecasts?
    - We typically focus on the "coverage" (do observations fall within the prediction intervals?), how far inside/outside observations are, and how wide the intervals themselves are
    - There are a few common evaluation metrics:
        - Quantile loss (this was used in the recent M5 competition)
        - Winkler score
        - Mean Interval Score (this was used in the M4 competition)
        - Continuous Ranked Probability Score

### Anomaly Detection

- Anomalies/Outliers are observations that are very different from the majority of the observations in the time series
    - They may be errors (human error, measurement error, machine error, etc.), or they may just be unusual or unique observations
    - There are countless methods for detecting outlier and only some are discussed here
    - But outlier detection is very field-specific and it takes some expert knowledge and interpretation to identify an outlier in the data
- Global vs Local Outliers
    - Global outliers: A data point with its value is far outside of the entirety of the data set (e.g., billionaires)
    - Local/Contextual outliers: A data point is considered a contextual outlier if its value significantly deviates from the rest the data points in the same context. (e.g., earning 50K income in a developing countries)

#### Rolling Median
1. Subtract a rolling median from the data (with suitable window size)
2. Calculate the s.d. of residuals $\hat{\sigma}_r$
3. Assume normally distributed residuals, identify outliers as $\pm1.96\times\hat{\sigma}_r$ (outside the 95% confidence interval)

#### STL Decomposition
- Seasonal Trend Decomposition with Loess
    - LOESS: locally estimated scatterplot smoothing
    - LOESS has a weighted version
- This approach mimics what happens is the R package `tsoutliers()` function of the R `forecast` package
- Methodology:
    1. Decompose data to find residuals:
        - Non-seasonal data: use a loess curve
        - Seasonal data: do STL decomposition
    2. Calculate $q_{0.1}$ and $q_{0.9}$ of the residuals
    3. Identify outliers as $\pm 2 \times (q_{0.9}-q_{0.1})$

#### Model-based
1. Fit a model to your data (e.g., ARIMA, ETS, machine learning, etc)
2. Identify outliers as significant deviations from model predictions
    - We can use the `model.get_prediction(start=1).summary_frame()` to get the in-sample prediction statistics

#### ML Methods
- Many supervised and even unsupervised approaches available and here are some common packages:
    - [pyod](https://pyod.readthedocs.io/en/latest/)
    - [sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
    - [luminaire](https://zillow.github.io/luminaire/)
    - [sklyline](https://earthgecko-skyline.readthedocs.io/en/latest/)
    - And many more...
- Isolation Forest
    - Like any tree ensemble method, isolation forest is built on the basis of decision trees.
        - In these trees, partitions are created by first randomly selecting a feature and then selecting a random split value between the minimum and maximum value of the selected feature (kinda like random forest, but this method is unsupervised)
    - Outliers are less frequent than regular observations and have more extreme values.
        - Hence when you use isolation forest for partition data points, outliers should be identified closer to the root of the tree (shorter average path length)
        - If the score is close to 1 it might be an anomaly and if it is much smaller than 0.5 it would be normal
        - If all scores are close to 0.5 then the entire sample does not seem to have distinct anomalies
    - In the course note [(link to course note)](https://pages.github.ubc.ca/MDS-2021-22/DSCI_574_spat-temp-mod_students/lectures/lecture5_uncertainty.html#isolation-forest) there is an example of sklearn's [IsolationForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html#sklearn.ensemble.IsolationForest) (a tree-based algorithm). 
        - Note that the `contamination` function affects how many outliers will be detected, and is something that's usually based on historical data or tuned with a train/valid/test split; tuning is skipped in the example
- K-nearest neighbor
    - For each observation, compute the average distance to its k-nearest neighbors
    - The largest average distance values tend to be outliers
    - Three kNN detectors are supported: 
      - largest: use the distance to the kth neighbor as the outlier score 
      - mean: use the average of all k neighbors as the outlier score 
      - median: use the median of the distance to k neighbors as the outlier score

### Imputation

- Imputation refers to the filling of missing values or outliers
    - Anomaly detection = detection of outliers/unusual data; Imputation = filling missing values/outliers
    - Sometimes missing values/outlier actually provide useful information about the process that produced the data, so you should be careful when imputing them
    - However, if you do have genuine errors in the data then imputing them can make the model-building process and forecasting better
- There are a few over-arching techniques we can use for imputation:
    1. Remove (`.dropna()`)
        - Generally a bad idea for time-series data
    2. Fill manually based on some expert-interpreted values (`.fillna()`)
    3. Fill naively using, e.g., mean, median, last observed value, random value
    4. Fill based on a rolling statistic, e.g., moving mean or median
    5. Polynomial interpolation (linear is often a good place to start unless you have a good reason to use a higher order)
    6. Fill based on temporal dependence ("temporal imputation"), i.e., use the same value from the same period last season, or the average of all periods in past seasons (e.g., to fill a January, use last January, or the average of all past January's)
    7. Fill with model fitted values
    8. More advanced methods leveraging multiple imputation, expectation maximization, etc. `fancyimpute` was the go-to package for advanced imputation in Python for many years, but now much of its core functionality has been included in `sklearn`, mostly in the `IterativeImputer` class. A common algorithm `MICE` is also [available in `statsmodels`](https://www.statsmodels.org/stable/generated/statsmodels.imputation.mice.MICE.html)
- For advanced reading on imputation, see [Flexible Imputation of Missing Data](https://stefvanbuuren.name/fimd/)

## Advanced Forecasting Modelling

### Deep Learning with Time Series

#### Basic Neural Network

- We wrangle some features from our time series and use those to train a network.
- Pros of using neural network rather than basic classical models:
    - They can model complex non-linear relationships.
    - They can easily do classification.
    - They can learn seasonality and patterns in the data and be effective when you have no expert knowledge of the data.
    - Easily incorporate additional explanatory variables (not just lags).
    - Some models, like CNNs and LSTMs can preserve the sequential nature of the series in clever ways to improve predictive power
- **Major concern** with basic neural network is that the *order of features does not really matter*
    - The important thing about time series data comparing to other data is its temporal structure, and if we are using basic neural network we are more or less throwing away this important characteristic

#### Convolutional Neural Network

- In previous block we learned that with CNN we can retain the image structure effectively and we would like to apply the similar logic here with time series data.
    - With image data, we use the `pytorch` class `torch.nn.Conv2d()`. 
    - With time series data, we use `torch.nn.Conv1d()`. 
    - But these layers are really doing the same thing, just in a different dimensionality.
- For example, rather than an image, we are now taking in some sequence of values and passing our filters over them to generate "filtered" versions of the series that focus on particular patterns/elements 
    - (some might highlight noise, some might smooth, some might highlight seasonality, etc.)
- The input data to our model should be shape `(batch size x features x sequence length)`:
    - `batch size` is up to you, as usual;
    - `features` is the number of features we have, for a univariate time series, that's 1.
    - `sequence length` is the length of the segment of time series we wish to feed in
        - this is something that can be tuned
        - if there's seasonality, usually set it to one or two times the seasonal period
- We first need to prepare our time series for our network. Our features will be sequences of size `sequence length` and our target will be the next value in the sequence (i.e., the one we wish to predict with our sequence).
- There are tons of variations we can make here:
    - Single input -> single output (we did this in the first example)
    - Multiple inputs -> single output
    - Single input -> multiple outputs
    - Multiple inputs -> multiple outputs (there is an example in that section)

#### Recurrent Neural Network
- CNN works reasonably well as it retains the structure of the data; however it has a new issue as it does not have any memory
    - values in the past have very limited influence on the values in the present or the recent past
    - remember that our time series data has dependency between historical and current data
- RNN comes in and splits the whole input data up and process one time-step at a time (i.e., sequentially). 
    - We can then feed the output of each time-step into the next as "memory" as part of the input for the next time-step
- The problem with basic RNN is that it only has short memory
    - Thus we build LSTM cells that pass through the long term "memory" together with the short term "memory" mentioned previously in basic RNN
        - Note that in LSTM all the LSTM components are identical. They share the same weights and the same architecture
        - The core idea is to combine the current input with the long-term and short-term memory to provide an output, a new short-term memory, and update the long-term memory
    - The architecture of an LSTM cell mentioned in the course note is fairly arbitrary - it has been found to work well in practice but there are plenty of variants of it. A popular one is GRU.
    - In some cases, the LSTM is "stateless" as the "memory" between batches are not being kept and carried through, but there might be cases where we have shorter time series and we would like the states to persist between batches, hence building a "stated" LSTM

### Useful Packages
- Facebook's Prophet
    - [Facebook's Prophet](https://facebook.github.io/prophet/) is another popular model for time series forecasting
    - It's actually not that complex of model, it basically fits non-linear regressions at various seasonalities (you can read the paper [here](https://peerj.com/preprints/3190/)). I like to think of it as a bit of a Frankenstein of classical time series methods we've seen like decomposition, regresion, exponential smoothing, etc.
    - Prophet can be useful for modelling data with complex/multiple seasonality, changing trends and holiday effects, and aims to automate and speed up model fitting as much as possible to help with forecasting data at scale.
    - Prophet was originally developed for modelling daily data. It has since expanded to include longer periods (weekly, monthly, yearly, etc.). 
        - There's plenty of comparisions out there of Prophet vs ARIMA vs ETS vs machine learning - but as always, there's no free lunch.
- Machine Learning based
    - GluonTS
        - GluonTS is a time series package built on the [MXNet](https://mxnet.incubator.apache.org/versions/1.8.0/) deep-learning framework (an alternative to `pytorch`)
        - It has a simple and easy to use Python API with plenty of ready-to-train state of the are deep learning models
        - Other popular models available in `gluonts` include:
            - [DeepAR](https://arxiv.org/abs/1704.04110)
            - [DeepVAR](https://arxiv.org/abs/1910.03002)
            - [GPVAR](https://arxiv.org/abs/1910.03002)
            - [DeepState](https://papers.nips.cc/paper/2018/hash/5cf68969fb67aa6082363a6d4e6468e2-Abstract.html)
            - [LSTNet](https://arxiv.org/abs/1703.07015)
            - [N-Beats](https://arxiv.org/abs/1905.10437) 
    - [pytorch-forecasting](https://github.com/jdb78/pytorch-forecasting) is a relatively new package released late 2020 that aims to provide a high-level API for neural network based time series modelling
        - Comparing to `gluonts`, they claim that `pytorch-forecasting` has the advantage of being built on `pytorch` which is more popular than `MXNet` and that the interface will be simpler than `gluonts`
    - Others
        - Two other popular libraries for time series forecasting with machine learning and an `sklearn` interface:
            1. [sktime](https://github.com/alan-turing-institute/sktime)
            2. tslearn
        - But there are [plenty of other libraries](https://www.sktime.org/en/latest/related_software.html#machine-learning) out there too
- R
    - Time series packages in R have recently been getting a huge upgrade in the form of the [tidyverts](https://tidyverts.org/) ecosystem
    - As the name suggests, this ecosystem is made to work nicely with tibbles and the [tidyverse](https://www.tidyverse.org/)!
    - The `tidyverts` ecosystem is under active development, but it contains all the same time series functionality we've been using in `statsmodels` and other packages

### Other advanced tools and knowledge

- **Heirachical/grouped time series**:
    - Time series that have multiple levels. For example we may wish to model unemployment in Australia on a per-country, and per-state level (Australia has 6 states and 2 territories)
    - Bottom-up approach: model the lowest level first (per-state unemployment) and then sum to produce higher levels (country unemployment)
    - Top-down approach: model the highest level first (country unemployment) and then proportion down to lower levels (per-state unemployment)
- **Multiple seasonality**:
    - Decompose/model independently
    - Decompose/model simultaneously using e.g., [prophet](https://facebook.github.io/prophet/), or [statsmodels](https://www.statsmodels.org/devel/examples/notebooks/generated/statespace_seasonal.html)
- **Multivariate series**:
    - We actually saw how to easily forecast multivariate series in Lecture 4 and earlier this lecture using machine learning
    - The idea is to allow the series to interact and affect each others' values
    - Another popular model for this, which is a generalisation of autoregression is [Vector Autoregression](https://www.statsmodels.org/dev/vector_ar.html)
- **Explanatory variables**:
    - Very easy to add a features to machine learning models
    - ARIMA models also support this, usually in the form of "[regression with ARIMA errors](https://robjhyndman.com/hyndsight/arimax/)", which basically means, fit a linear regression to your data and then model the residual with an ARIMA model. Try it out by using the `exog` parameters of `statsmodels` `ARIMA()` class.
- **Time series classification**:
    - There are other options than the normal machine learning approach, like hidden Markov models, you can also try Googling: "discrete time series forecasting"

## Time Series Analysis Summary
- Key takeaways:
    1. Time series data is not independent, it contains temporal dependence. The main patterns we look for are:
        - Trend
        - Seasonality
        - Cyclicity
        - Short-term autocorrelation
    2. The temporal patterns in a series help us choose suitable models. There are 4 main classes of models we can use to model time series:
        - Baseline models (naive, average, drift, etc.)
        - Exponential smoothing models 
        - ARIMA
        - Machine learning
    3. Each of the above classes model time series in different ways. There's no free lunch, but ensembles (combinations) of different models can work well. The statistical models remain popular in industry because of their speed, ease-of-use, and interpretibility. The machine learning approaches take much more effort to build, maintain, and forecast with - but they show potential!
    4. My workflow when presented with a new time series:
        - Understand it. Plot it. Decompose it. What signals are present? What models might be suitable here?
        - Split into train, validation, and test sets (or just train and test if you intend on doing cross-validation in the training set).
        - Baseline and statistical models are the first stop. ETS and ARIMA are the workhorses of time series modelling, but don't discount the potential of basline methods!
        - If you're looking for more performance, delve into machine learning. In particular, this allows us to easily and effectively incorporate explanatory variables and model highly complex functions.
        - If you've got the time and the patience, say hello to CNNs and LSTMs. They are complex, and take a long time to develop, but they show great potential. To me these methods try to interpret the time series for you, they engineer the features and identify the seasonality. In my experience, I've struggled to build neural networks that consistently outperform the classic statistical methods. But in saying that, they offer the ability to get good predictive performance without expert knowledge of the field or data you are forecasting.
    5. Always remember. We are forecasting the future - nobody knows what's going to happen. In that spirit, I prefer to err on the side of parsimony (simplicity).